# Cdiscount ConvNet with Keras

In [1]:
import os
import glob
from scipy import misc
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm_notebook as tqdmn

Using TensorFlow backend.


## Import the data into numpy array

In [2]:
img_width = 180
img_height = 180
n_channel = 3
train_folder = 'input/train'
validation_folder = 'input/validation'
test_folder = 'input/test'

class_list = os.listdir(train_folder)
n_classes = len(class_list)
n_train_examples = 12371293 #adapt
n_test_examples = 3095080



In [3]:
N_TRAINING_PER_CLASS

NameError: name 'N_TRAINING_PER_CLASS' is not defined

In [3]:
K.set_image_dim_ordering('th')
if K.image_data_format() == 'channels_first':
    input_shape = (n_channel, img_width, img_height)
else:
    input_shape = (img_width, img_height, n_channel)
    
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5270))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [4]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'input/train',  
        target_size=(img_height, img_width), 
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        'input/validation',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

Found 8659498 images belonging to 5270 classes.
Found 3711795 images belonging to 5270 classes.


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13758465111039610198
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 1392836608
locality {
  bus_id: 1
}
incarnation: 16953869981788945744
physical_device_desc: "device: 0, name: GeForce 940MX, pci bus id: 0000:02:00.0"
]


In [6]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=test_generator,
        validation_steps=800 // batch_size)
model.save_weights('convnet_weights_1.h5')

Epoch 1/50
125/125 [==============================] - 123s - loss: 8.0322 - acc: 0.0045 - val_loss: 7.3822 - val_acc: 0.0088
Epoch 2/50
125/125 [==============================] - 39s - loss: 7.3835 - acc: 0.0060 - val_loss: 7.2795 - val_acc: 0.0050
Epoch 3/50
125/125 [==============================] - 39s - loss: 7.2480 - acc: 0.0085 - val_loss: 7.2238 - val_acc: 0.0050
Epoch 4/50
125/125 [==============================] - 41s - loss: 7.1844 - acc: 0.0070 - val_loss: 7.1324 - val_acc: 0.0075
Epoch 5/50
125/125 [==============================] - 41s - loss: 7.2021 - acc: 0.0070 - val_loss: 7.0443 - val_acc: 0.0112
Epoch 6/50
125/125 [==============================] - 41s - loss: 7.1427 - acc: 0.0055 - val_loss: 7.1187 - val_acc: 0.0063
Epoch 7/50
125/125 [==============================] - 41s - loss: 7.0920 - acc: 0.0090 - val_loss: 7.0540 - val_acc: 0.0075
Epoch 8/50
125/125 [==============================] - 40s - loss: 7.1807 - acc: 0.0065 - val_loss: 6.9809 - val_acc: 0.0050
Epoch 9

In [10]:
scores = model.evaluate_generator(test_generator, 2000 // batch_size)

In [11]:
print("Accuracy = ", scores[1])

Accuracy =  0.0075
